In [51]:
from PIL import Image
from time import time_ns
import math
import numpy as np
import os
import ipywidgets as widgets

In [52]:
def get_all_images():
  fileDict = {}
  for img_name in  os.listdir("./input"):
    img_file = Image.open('input/' + img_name, 'r').convert('RGB')
    fileDict.update({img_name: img_file})
  return fileDict

In [54]:
def binarization(image):
  result = Image.new('L', (image.width, image.height))

  for x in range(result.width):  #цвет в полутон
    for y in range(result.height):
      pixel = image.getpixel((x, y))
      new_pixel = int(round(pixel[0] * 0.3 + pixel[1] * 0.59 + pixel[2] * 0.11))
      result.putpixel((x, y), new_pixel)

  return result

In [55]:
operator_prewitt_x = np.array([[-1, -1, -1], 
                               [ 0,  0,  0],
                               [ 1,  1,  1]])
operator_prewitt_y = np.array([[-1, 0, 1], 
                               [-1, 0, 1],
                               [-1, 0, 1]])

In [56]:
s = widgets.FloatSlider(
  value=2.5,
  min=0,
  max=10.0,
  step=0.5,
  description='Test:',
  disabled=False,
  continuous_update=True,
  orientation='horizontal',
  readout=True,
  readout_format='.1f',
)
s

FloatSlider(value=2.5, description='Test:', max=10.0, readout_format='.1f', step=0.5)

In [57]:
def contour_selection(image, operator_x, operator_y, window_size):
  width, height = image.size
  S = s.value
  print(S)
  gray_arr = np.asarray(image, dtype=np.uint8)
  
  G_x_matrix = np.zeros(shape=(height, width))
  G_y_matrix = np.zeros(shape=(height, width))
  G_matrix = np.zeros(shape=(height, width))
  
  padded_gray_arr = np.pad(gray_arr, ((1, 1), (1, 1)), mode='constant')  #однопиксельные отступы вокруг, чтобы градиент на краях был в норме
  for i in range(1, height+1): 
    for j in range(1, width+1): 
      i_min = max(i - window_size // 2, 0)
      i_max = min(i + window_size // 2, height+1)
      j_min = max(j - window_size // 2, 0)
      j_max = min(j + window_size // 2, width+1)
      
      window_matrix = padded_gray_arr[i_min:i_max+1, j_min:j_max+1] #матрица значений для окна свертки
      
      G_x = np.sum(operator_x * window_matrix) #компоненты градиента
      G_y = np.sum(operator_y * window_matrix)
      G = np.abs(G_x) + np.abs(G_y)

      G_x_matrix[i-1, j-1] = int(round(G_x))
      G_y_matrix[i-1, j-1] = int(round(G_y))
      G_matrix[i-1, j-1] = int(round(G))
        
  G_x_matrix_normalized = np.abs(G_x_matrix / np.max(G_x_matrix) * 255) #не выходить за ограду от 0 до 255!!
  G_y_matrix_normalized = np.abs(G_y_matrix / np.max(G_y_matrix) * 255)
  G_matrix_normalized = G_matrix / np.max(G_matrix) * 255
  
  t = np.mean(G_matrix_normalized) * S   #порог бинаризации(среднее на S)

  gXImg = Image.fromarray(G_x_matrix_normalized.astype(np.uint8), mode='L')
  gYImg = Image.fromarray(G_y_matrix_normalized.astype(np.uint8), mode='L')

  g_img = Image.fromarray(G_matrix_normalized.astype(np.uint8), mode='L')
  binary_img = Image.fromarray((G_matrix_normalized > t).astype(np.uint8) * 255, mode = 'L') # порог бинаризации к суммарной матрице, выше - снежок, ниже - уголек

  return gXImg, gYImg, g_img, binary_img


In [58]:
def proceed_contour(operator_x, operator_y, window_size: int):
  images = get_all_images()

  for (image_name, image) in images.items():
    grayscale_image = binarization(image)
    g_x_img, g_y_img, g_img, binary_img = contour_selection(grayscale_image, operator_x, operator_y, window_size)

    g_x_img.save('output/' + image_name[:-4] + '_x.processed.png')
    g_y_img.save('output/' + image_name[:-4] + '_y.processed.png')
    g_img.save('output/' + image_name[:-4] + '.processed.png')
    binary_img.save('output/' + image_name[:-4] + '_binary.processed.png')
    grayscale_image.save('output/' + image_name[:-4] + '_grayscale.processed.png')

In [59]:
def main():
  proceed_contour(operator_prewitt_x, operator_prewitt_y, 3)

In [61]:
if __name__ == '__main__':
  main()


4.0


KeyboardInterrupt: 